In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
#from scipy import interp, arange, exp
from ccfj import GetStationPairs
import ccfj
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor
import shutil
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy import stats

In [ ]:
import sys
sys.path.append('../tools_F-J')
from toollib_voro import PointinPolygon
sys.path.append(r'../tools_F-J/')
from toollib_standard import mathlib

In [ ]:
with open('a-project_voro.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartrition/output_repar_03-01/'               # Harmon server
name_project

In [ ]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

In [ ]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [ ]:
dir_stack = dir_project+info_basic['rdir_stack']
dir_CC = dir_CC_workspace+info_basic['rdir_CC']
dir_ds =  dir_project + info_basic['rdir_ds']
stalistname_all = info_basic['stalistname_all']

In [ ]:
def Pairs(sta):
    p = []
    nsta = len(sta)
    for ii in range(nsta):
        for jj in range(ii+1,nsta):
            p.append([sta[ii],sta[jj]])
    return p
def cal_indx(pair,nsta):
    indx = int(pair[0]*(2*nsta-pair[0]-1)/2+pair[1]-pair[0]-1)
    return indx

In [ ]:
def time_window_filter(t,ncfst0,r,v_min,t0,a):
    ncfst = ncfst0.copy()
    for i in range(len(ncfst)):
        tag = r[i]/v_min
        #print(t0,tag)
        t1 = t[t>-tag-t0][t[t>-tag-t0]< tag+t0]
        start = np.where(t == t1[0])[0][0]
        end = np.where(t == t1[-1])[0][0]
        ncfst[i][start:end+1]= ncfst[i][start:end+1]* np.exp(-a*np.abs((tag-np.abs(t1))))
    return ncfst

In [ ]:
def singleVoro(indx0,outname,xvyv):
    global ncfs
    global f0
    global c
    global nf
    global r
    global count
    global dir_ds
    global nsta

    global t0
    global a
    global v_tag

    filename = os.path.join(dir_ds,outname+'.h5')
    if os.path.exists(filename):
        os.remove(filename)
    
    subpairs = Pairs(indx0)
    indx1 = [cal_indx(pair,nsta) for pair in subpairs]
    ncfsi = ncfs[indx1,:]
    counti = count[indx1]
    ri = r[indx1]
    indx = np.argsort(ri)
    ri = ri[indx]
    ncfsi = ncfsi[indx,:]
    counti = counti[indx]
    ncfsi = ncfsi[counti!=0]
    ri = ri[counti!=0]
    counti = counti[counti!=0]
    for i in range(len(ri)):
        ncfsi[i,:] = ncfsi[i,:]/counti[i]

    dt = 1/np.max(f0)
    t = (np.linspace(-len(f0)-1,len(f0)-1,2*(len(f0)-1))+0.5)*dt/2
    ncfst_linear = mathlib.freq_time(ncfsi)
    ncfst1 = time_window_filter(t,ncfst_linear,r,v_tag,t0,a)
    ncfs_sum_remove = mathlib.time_freq(ncfst1)
    
    ds_11 = ccfj.fj_noise(np.real(ncfsi),ri,c,f0,itype=1,func=1)
    ds_22 = ccfj.fj_noise(np.real(ncfs_sum_remove),ri,c,f0,itype=1,func=1)

    ds_linear = np.array(ds_11).reshape(1,np.shape(ds_11)[0],np.shape(ds_11)[1])
    ds_remove = np.array(ds_22).reshape(1,np.shape(ds_22)[0],np.shape(ds_22)[1])

    h5file=h5py.File(filename,'w')
    h5file.create_dataset('ds_linear',data=ds_linear)
    h5file.create_dataset('ds_remove',data=ds_remove)
    h5file.create_dataset('c',data=c)
    h5file.create_dataset('f',data=f0)
    h5file.create_dataset('indx',data=indx0)
    h5file.create_dataset('xvyv',data=xvyv)
    h5file.close()

In [ ]:
stainfo = pd.read_excel(stalistname_all)
nsta = len(stainfo.iloc[:,0])
staloc = np.asarray([stainfo['longitude'],stainfo['latitude']]).T
# The KDE of the stations' locations
kernel = stats.gaussian_kde(staloc.T)

In [ ]:
ncffile = h5py.File( dir_CC+ 'gather_all.h5','r')
ncfs = ncffile['ncfs'][:]
f0 = ncffile['f'][:]
count = ncffile['count'][:]
r = ncffile['r'][:]
ncffile.close()
nf = info_basic['nf']

In [ ]:
c_min = 0.200
c_max = 2
c_num = 800
info_basic['c_min'] = c_min
info_basic['c_max'] = c_max
info_basic['c_num'] = c_num
c = np.linspace(c_min,c_max,c_num)


In [ ]:
# VoroTomo Partition times
n_voro = 100
# number of Voro cells  range
kmin = 10
kmax = 20

# Set a larger range of the stations to facilitate the calculation of Voronoi cells
# the points0 are not used in the inner Voronoi cells generating
Radius = 10 
x0 = np.mean(staloc[:,0])
y0 = np.mean(staloc[:,1])
minx = min(staloc[:,0]);maxx = max(staloc[:,0])
miny = min(staloc[:,1]);maxy = max(staloc[:,1])
points0 = []
for i in range(36):
    points0.append([x0+np.cos(i/18*np.pi),y0+np.sin(i/18*np.pi)])
points0=np.asarray(points0)

In [ ]:
info_basic['voro_n'] = n_voro
info_basic['voro_kmin'] = kmin
info_basic['voro_kmax'] = kmax
info_basic['voro_Radius'] = Radius

In [ ]:
v_tag = 2
a = 100
t0 = 0.01

In [ ]:
# Starting Voronoi Partitioning
key_subworks = []
for ii in range(n_voro):
    print('start:' + str(ii))
    # random the number of Voronoi cells
    k = np.random.randint(kmin,kmax)
    # Using the KDE PDF to generate Voronoi cells
    points = kernel.resample(k).T 
    # Using the Uniform distribution to generate Voronoi cells
    '''
    points = np.random.rand(k,2)
    points[:,0] = points[:,0]*(maxx-minx) + minx
    points[:,1] = points[:,1]*(maxy-miny) + miny
    '''
    # Voronoi Partition
    points = np.concatenate((points,points0))
    vor = Voronoi(points)
    areas = []
    for j in range(k):
        xv = vor.vertices[vor.regions[vor.point_region[j]],0]
        yv = vor.vertices[vor.regions[vor.point_region[j]],1]
        _in,_on = PointinPolygon.inpolygon(staloc[:,0],staloc[:,1],xv,yv)
        indx = np.array([i for i,x in enumerate(_in) if x])
        if len(indx)>10:
            areas.append([indx,np.array([xv,yv])])
    vor.close()
    for j in range(len(areas)):
        key_subwork = 'vor'+str(ii)+'_'+str(j)
        key_subworks.append(key_subwork)
        print(ii,j,len(areas[j][0]))
        # calculate the F-J spectrum
        singleVoro(areas[j][0],key_subwork,areas[j][1])

In [ ]:
info_basic['key_subworks'] = key_subworks
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)